In [1]:
# this sets up basic packages
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None 
import astropy.units as u
import astropy.cosmology.units as cu

# this sets up matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# this sets up astropy
from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs.utils import pixel_to_skycoord
from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, Angle, match_coordinates_sky, Distance
from astropy.cosmology import Planck15 as cosmo
from astropy.table import Table

from regions import Regions, CircleSkyRegion

In [2]:
# first, import the file of neighbor data, which is a csv
df = pd.read_csv('neighbor_data.csv')

First, we deal with the space-based observations from Hollis's catalog.

In [4]:
# this reads in all the COSMOS-Web measurements in different bands
f_cweb_cols = [col for col in df.columns if 'f_auto' in col]
e_cweb_cols = [col for col in df.columns if 'e_auto' in col]

# this saves the names of the COSMOS-Web bands
space_band_names = []
for header in f_cweb_cols:
    name = header[-5:] # this gets the band name, which are the last 5 characters of the column header
    space_band_names.append(name)

# check how many bands we've had for the first time
print(space_band_names)

# this reads in all the PRIMER measurements in different bands
f_primer_cols = [col for col in df.columns if 'f_primer_auto' in col]
e_primer_cols = [col for col in df.columns if 'e_primer_auto' in col]

# this saves the names of the PRIMER bands
for header in f_primer_cols:
    name = header[-5:] # this gets the band name, which are the last 5 characters of the column header
    if name not in space_band_names:
        space_band_names.append(name)

# check how many bands we've had for the second time
print(space_band_names)

['f814w', 'f115w', 'f150w', 'f277w', 'f444w', 'f770w']
['f814w', 'f115w', 'f150w', 'f277w', 'f444w', 'f770w', 'f606w', 'f090w', 'f200w', 'f356w', 'f410m']


In [5]:
# now, we make a simple 5sigma to 3sigma conversion. this is for later usage
def convert_3sigma(m_5sig):
    m_3sig = m_5sig - np.log10(5) + np.log10(3)
    return m_3sig

In [6]:
# now, we make a list of the 3sigma for each space band. the sources are 
# Weaver et al (2020): arxiv.org/pdf/2110.13923, and Casey et al (2023): iopscience.iop.org/article/10.3847/1538-4357/acc2bc/pdf.
# note that for values from Casey et al, they're actually 5sigma and have to be recalculated as 3sigma.
space_3sigma_list = [27.8, convert_3sigma(27.13), convert_3sigma(27.35), convert_3sigma(27.99), convert_3sigma(27.83), 
                     convert_3sigma(25.70), convert_3sigma(27.8), convert_3sigma(27.6), convert_3sigma(28.2), 
                     convert_3sigma(28.4), convert_3sigma(27.7)] # 

# now we combine everything into a dictionary so it's easier to sort through later
space_3sigma_dict = {space_band_names[i]: space_3sigma_list[i] for i in range(len(space_band_names))}

In [7]:
print(space_3sigma_dict)

{'f814w': 27.8, 'f115w': 26.908151250383643, 'f150w': 27.128151250383645, 'f277w': 27.768151250383642, 'f444w': 27.608151250383642, 'f770w': 25.478151250383643, 'f606w': 27.578151250383645, 'f090w': 27.378151250383645, 'f200w': 27.978151250383643, 'f356w': 28.178151250383642, 'f410m': 27.478151250383643}


Now let's do the dreaded for-loop that goes through the data and gets the best fluxes and flux errors.

In [9]:
# THIS IS FOR THE SPACE-BASED DATA

fluxes = []
errors = []

for name in space_band_names:    
    ### first, read in the headers of the bands that are available in both COSMOS-Web and PRIMER
    cweb_flux_header = [header for header in f_cweb_cols if name in header]
    cweb_err_header = [header for header in e_cweb_cols if name in header]
    primer_flux_header = [header for header in f_primer_cols if name in header]
    primer_err_header = [header for header in e_primer_cols if name in header]

    ### next, load in the 3sigmas from our dictionary. remember that 
    ### our 3sigma values, which are given in magnitudes, need to be converted into uJy.
    three_sigma = (10**((23.9-space_3sigma_dict[name])/2.5))
    
    ### next, check to see if the bands with these headers actually exist for BOTH COSMOS-Web AND PRIMER
    if len(cweb_flux_header) != 0 and len(primer_flux_header) != 0:
        # next, check to see how much of the data under these headers is actually NaN.
        # if they're NaN, set them to -99.0, so CIGALE will ignore them later on.
        cweb_flux = df[cweb_flux_header[0]].replace(np.nan, -99.0)
        cweb_err = df[cweb_err_header[0]].replace(np.nan, -99.0)
        primer_flux = df[primer_flux_header[0]].replace(np.nan, -99.0)
        primer_err = df[primer_err_header[0]].replace(np.nan, -99.0)

        # next, check for values that are less than zero. these values will be set to -99.0 as well.
        cweb_flux[cweb_flux <= 0] = 0 
        cweb_err[cweb_err <= 0] = three_sigma
        primer_flux[primer_flux <= 0] = 0
        primer_err[primer_err <= 0] = three_sigma
        
        # then we calculate and compare the signal-to-noise ratio (SNR), 
        # then take the one (PRIMER vs COSMOS-Web) with the greater SNR.
        cweb_snr = cweb_flux / cweb_err
        primer_snr = primer_flux / primer_err

        # create an empty array to save the flux and flux error with better SNR
        better_flux = np.zeros(np.size(cweb_snr))
        better_err = np.zeros(np.size(cweb_snr))
        better_snr = np.zeros(np.size(cweb_snr))
        for i in range(np.size(cweb_snr)):
            if primer_snr[i] > cweb_snr[i]: # if PRIMER's SNR is greater than CWeb's, then keep PRIMER's flux, error & SNR
                better_flux[i] = primer_flux[i]
                better_err[i] = primer_err[i]
                better_snr[i] = primer_snr[i]
            else: # else if PRIMER's SNR is smaller than CWeb's, then keep CWeb's flux, error & SNR
                better_flux[i] = cweb_flux[i]
                better_err[i] = cweb_err[i]
                better_snr[i] = cweb_snr[i]

    ### now, check the cases where there is CWeb data but NO PRIMER data
    elif len(cweb_flux_header) != 0:
        cweb_flux = df[cweb_flux_header[0]].replace(np.nan, -99.0)
        cweb_err = df[cweb_err_header[0]].replace(np.nan, -99.0)

        # then we calculate the SNR
        better_flux = cweb_flux
        better_err = cweb_err
        better_snr = cweb_flux / cweb_err

    ### finally, check the cases where there is PRIMER data but NO CWeb data
    else:
        primer_flux = df[primer_flux_header[0]].replace(np.nan, -99.0)
        primer_err = df[primer_err_header[0]].replace(np.nan, -99.0)

        # then we calculate the SNR
        better_flux = primer_flux
        better_err = primer_err
        better_snr = primer_flux / primer_err

    ### NOW, after gathering all this, we check for the final time to see if the SNR is less than or equal to 3.
    ### if yes, we'll set the error to be equal to the flux, and the flux to be equal to 0. we have to do this
    ### because CIGALE likes to make weird estimational violations when it comes to large upper limits.
    for i in range(np.size(better_snr)):
        SNR = better_snr[i]
        if SNR <= 3:
            better_err[i] = three_sigma
            better_flux[i] = 0

    ### finally, we convert all the fluxes and errors 
    ### from μJy (the unit used in Hollis's table) to mJy (the unit that CIGALE takes)
    better_flux = better_flux / 1e3
    better_err = better_err / 1e3

    ### at last, we can escape this hellscape of a for-loop.
    ### save the fluxes and errors into a list so we can read later.
    fluxes.append(better_flux)
    errors.append(better_err)

After this trial and tribulation of a for-loop, let's do a sanity check!

Then, at last, we deal with the ground-based observations from Marko.

In [12]:
# this reads in all the ground-based data from Marko
f_ground_cols = [col for col in df.columns if col.startswith('FLUX_MODEL')]
e_ground_cols = [col for col in df.columns if col.startswith('FLUX_ERR_MODEL')]

# this adds all the bands in Marko's ground-based catalog
ground_band_names = []
for header in f_ground_cols:
    name = header[11:] # this gets the band name
    # because CIGALE doesn't have all the bands of each observatory,
    # we'll have to do some exclusion
    if name.startswith('CFHT'):
        ground_band_names.append(name)
    elif name.startswith('HSC') and len(name[7:]) < 2:
        ground_band_names.append(name)
    elif name.startswith('UVISTA') and len(name[7:]) < 3:
        ground_band_names.append(name)
    elif name.startswith('SC-IB') or name.startswith('SC-NB'):
        ground_band_names.append(name)
    elif name.startswith('IRAC') and int(name[-1]) < 4: # exclude IRAC-ch4 because there's no error column for this
        ground_band_names.append(name)

# check to see if we get the right bands
print(ground_band_names)

['CFHT-u', 'HSC-g', 'HSC-r', 'HSC-i', 'HSC-z', 'HSC-y', 'UVISTA-Y', 'UVISTA-J', 'UVISTA-H', 'UVISTA-Ks', 'SC-IB427', 'SC-IB505', 'SC-IB574', 'SC-IB709', 'SC-IB827', 'SC-NB711', 'SC-NB816', 'IRAC-ch1', 'IRAC-ch2', 'IRAC-ch3']


In [13]:
# now, we make a list of the 3sigma for each ground band. the source is Weaver et al (2020): arxiv.org/pdf/2110.13923.
ground_3sigma_list = [27.2, 27.5, 27.2, 27.0, 26.6, 25.9, 26.1, 25.9, 25.5, 25.2, 
                    25.6, 25.6, 25.3, 25.4, 25.1, 24.9, 25.1, 25.7, 25.6, 22.6]

# now we combine everything into a dictionary so it's easier to sort through later
ground_3sigma_dict = {ground_band_names[i]: ground_3sigma_list[i] for i in range(len(ground_band_names))}

In [14]:
# THIS IS FOR THE GROUND-BASED DATA

for name in ground_band_names:
    ### first, read in the headers of the bands
    flux_header = [header for header in f_ground_cols if name in header]
    err_header = [header for header in e_ground_cols if name in header]

    ### next, we recalculate the data. there's this issue where, according to Arianna, the software used
    ### to measure Marko's fluxes has a bug where it claims to measure sources below the limit, 
    ### which is not possible. therefore, we'll pick the flux values that are below 3sigma survey limits
    ### and then set that to zero, and then set the corresponding flux errors to 3sigma.

    # first, load in the 3sigmas from our dictionary. remember that our 3sigma values, which came from
    # the COSMOS2020 paper, need to be converted from magnitudes into uJy.
    three_sigma = (10**((23.9-ground_3sigma_dict[name])/2.5))

    # next, reset the flux values below 3sigma to zero, and reset the errors there to 3sigma.
    flux = df[flux_header[0]]
    err = df[err_header[0]]
    flux[flux < three_sigma] = 0
    err[flux < three_sigma] = three_sigma

    # finally, check to see how much of the data under these headers is actually NaN.
    # if they're NaN, set them to -99.0, so CIGALE will ignore them later on.
    flux = flux.replace(np.nan, -99.0)
    err = err.replace(np.nan, -99.0)

    ### save the fluxes and errors into a list so we can read later.
    fluxes.append(flux)
    errors.append(err)

FINALLY, let's start preparing the contents for the dataframe that we'll later turn into a .txt file for CIGALE to read.

In [16]:
### first, we need the column names. 
# let's make a list for this. we have two known names: id and redshift.
column_names = ['id', 'redshift']

# next, we retrieve all the SPACE band names and make column names with them
for name in space_band_names:
    if name == 'f814w' or name == 'f606w': # these are to distinguish the HST bands
        column_names.append('hst.wfc.' + name.upper()) # make a column name for the flux
        column_names.append('hst.wfc.' + name.upper() + '_err') # make a column name for the error
    elif name == 'f770w': # this is to distinguish the MIRI band
        column_names.append('jwst.miri.' + name.upper()) # make a column name for the flux
        column_names.append('jwst.miri.' + name.upper() + '_err') # make a column name for the error
    
    else: # the rest are JWST bands
        column_names.append('jwst.nircam.' + name.upper()) # make a column name for the flux
        column_names.append('jwst.nircam.' + name.upper() + '_err') # make a column name for the error

print(column_names)

['id', 'redshift', 'hst.wfc.F814W', 'hst.wfc.F814W_err', 'jwst.nircam.F115W', 'jwst.nircam.F115W_err', 'jwst.nircam.F150W', 'jwst.nircam.F150W_err', 'jwst.nircam.F277W', 'jwst.nircam.F277W_err', 'jwst.nircam.F444W', 'jwst.nircam.F444W_err', 'jwst.miri.F770W', 'jwst.miri.F770W_err', 'hst.wfc.F606W', 'hst.wfc.F606W_err', 'jwst.nircam.F090W', 'jwst.nircam.F090W_err', 'jwst.nircam.F200W', 'jwst.nircam.F200W_err', 'jwst.nircam.F356W', 'jwst.nircam.F356W_err', 'jwst.nircam.F410M', 'jwst.nircam.F410M_err']


In [17]:
# next, we retrieve all the GROUND band names and make column names with them
for name in ground_band_names:
    if name.startswith('CFHT'):
        column_names.append('cfht.megacam.' + name[5:])
        column_names.append('cfht.megacam.' + name[5:] + '_err')
    elif name.startswith('HSC') and len(name[7:]) < 2:
        column_names.append('subaru.hsc.' + name[4:])
        column_names.append('subaru.hsc.' + name[4:] + '_err')
    elif name.startswith('UVISTA') and len(name[7:]) < 3:
        column_names.append('vista.vircam.' + name[7:])
        column_names.append('vista.vircam.' + name[7:] + '_err')
    elif name.startswith('SC-IB') or name.startswith('SC-NB'):
        column_names.append('subaru.suprime.' + name[3:])
        column_names.append('subaru.suprime.' + name[3:] + '_err')
    elif name.startswith('IRAC'):
        column_names.append('spitzer.irac.' + name[5:])
        column_names.append('spitzer.irac.' + name[5:] + '_err')

In [18]:
print(column_names)

['id', 'redshift', 'hst.wfc.F814W', 'hst.wfc.F814W_err', 'jwst.nircam.F115W', 'jwst.nircam.F115W_err', 'jwst.nircam.F150W', 'jwst.nircam.F150W_err', 'jwst.nircam.F277W', 'jwst.nircam.F277W_err', 'jwst.nircam.F444W', 'jwst.nircam.F444W_err', 'jwst.miri.F770W', 'jwst.miri.F770W_err', 'hst.wfc.F606W', 'hst.wfc.F606W_err', 'jwst.nircam.F090W', 'jwst.nircam.F090W_err', 'jwst.nircam.F200W', 'jwst.nircam.F200W_err', 'jwst.nircam.F356W', 'jwst.nircam.F356W_err', 'jwst.nircam.F410M', 'jwst.nircam.F410M_err', 'cfht.megacam.u', 'cfht.megacam.u_err', 'subaru.hsc.g', 'subaru.hsc.g_err', 'subaru.hsc.r', 'subaru.hsc.r_err', 'subaru.hsc.i', 'subaru.hsc.i_err', 'subaru.hsc.z', 'subaru.hsc.z_err', 'subaru.hsc.y', 'subaru.hsc.y_err', 'vista.vircam.Y', 'vista.vircam.Y_err', 'vista.vircam.J', 'vista.vircam.J_err', 'vista.vircam.H', 'vista.vircam.H_err', 'vista.vircam.Ks', 'vista.vircam.Ks_err', 'subaru.suprime.IB427', 'subaru.suprime.IB427_err', 'subaru.suprime.IB505', 'subaru.suprime.IB505_err', 'subaru.s

In [19]:
### finally, let's put everything into each of the columns! 
# to do this, we'll first create a dictionary.
mega_dict = {}

# next, we'll create an array of redshifts that are all set at -99.0.
# these are meant to be placeholders to CIGALE to fill in when it runs.
z_arr = np.full(np.size(fluxes[0]), -99.0)

# now we start populating the dictionary
for idx in range(len(column_names)-1):
    # FIRST, we add the id array, as defined above
    if idx == 0:
        mega_dict[column_names[idx]] = df['id']
    # NEXT, we add the redshift array, where all the values are set to -99.0
    elif idx == 1:
        mega_dict[column_names[idx]] = z_arr
    # FINALLY, we start adding the fluxes and errors, based on the band names. we know
    # the order of the band names in the band_names list and the column_names list are
    # the same, so one by one, we'll populate the next columns.
    else:
        if idx % 2 == 0:
            mega_dict[column_names[idx]] = fluxes[int(idx/2-1)]
            mega_dict[column_names[idx+1]] = errors[int(idx/2-1)]

cigale_df = pd.DataFrame(mega_dict)
print(cigale_df)

          id  redshift  hst.wfc.F814W  hst.wfc.F814W_err  jwst.nircam.F115W  \
0     756228     -99.0       0.000068           0.000010           0.000085   
1     756257     -99.0       0.000087           0.000009           0.000078   
2     756324     -99.0       0.000043           0.000009           0.000000   
3     756353     -99.0       0.000242           0.000014           0.000323   
4     756380     -99.0       0.000350           0.000024           0.000404   
...      ...       ...            ...                ...                ...   
1021  824190     -99.0       0.000039           0.000010           0.000000   
1022  824219     -99.0       0.000000           0.000028           0.000000   
1023  824230     -99.0       0.000010           0.000002           0.000000   
1024  824244     -99.0       0.000024           0.000004           0.000000   
1025  824265     -99.0       0.000000           0.000028           0.000000   

      jwst.nircam.F115W_err  jwst.nircam.F150W  jws

In [20]:
# after some checking, we know there's duplicates in the data table.
# this gets rid of the duplicates and keeps only one galaxy instead of one with many duplicates.
cigale_df_to_save = cigale_df[cigale_df.duplicated(keep='first')==False]

In [21]:
# this creates a big .txt file for CIGALE to run
cigale_df_to_save.to_csv('cigale_data.txt', sep='\t', index=False)

In [22]:
# this creates a big .csv file for later analysis
cigale_df_to_save.to_csv('post_correction_data.csv', index=False)

ELIMINATE AFTER VISUAL INSPECTIONS

In [24]:
nebular_rerun = np.array([659622, 756994, 661909, 642604, 614224, 614320, 614548, 614670, 
                 659534, 661661, 661662, 661988, 661989, 662092, 756380, 756650, 
                 757025, 759202, 781608, 781845, 784642, 784663, 823371, 759485, 
                 759140, 759207, 829990, 780261, 757435, 757212, 659693, 614256, 
                 641835, 641328, 614660, 614700], dtype='object')

In [25]:
postage_check = np.array([614453, 660158, 661580, 661675, 661906, 780995, 759290, 824112,
                         781056, 642340, 642380, 614114], dtype='object')

In [26]:
garbage = np.array([614134, 614257, 641889, 642233, 642502, 642581, 642605,
                    659467, 661883, 661884, 683294, 683364, 684046, 684102, 756594, 
                    757210, 782194, 782746, 783117, 783144, 783257, 784766, 823655, 823981, 830151, 680965], dtype='object')

In [27]:
# this takes out the rows that are in nebular_rerun
nebular = cigale_df_to_save.loc[cigale_df_to_save['id'].isin(nebular_rerun)]
nebular.to_csv('nebular_rerun_data.txt', sep='\t', index=False)

In [28]:
# this takes out the rows that are in postage_check
postage = cigale_df_to_save.loc[cigale_df_to_save['id'].isin(postage_check)]
postage.to_csv('postage_check_data.txt', sep='\t', index=False)

In [29]:
good_fits = np.array([756228, 756324, 756381, 756542, 756650, 756791, 756809, 756895,
       756931, 780235, 780261, 780365, 780483, 780489, 780511, 780592,
       780808, 780830, 780870, 780882, 780995, 781053, 781056, 757782,
       757931, 757934, 757952, 757967, 758035, 758076, 758096, 758165,
       782619, 782740, 782969, 783186, 783189, 783202, 783240, 804680,
       804711, 830212, 830422, 830622, 756960, 756994, 757025, 757035,
       757177, 757208, 757258, 757350, 757396, 757532, 780960, 781069,
       781337, 781608, 781836, 781863, 758845, 758998, 759022, 759045,
       759140, 759202, 759207, 759298, 759334, 759455, 759460, 759545,
       759592, 759715, 783764, 783888, 783941, 783964, 784079, 784159,
       784266, 784406, 784483, 784557, 784569, 784580, 784614, 784646,
       784663, 614117, 614284, 614318, 614521, 614660, 641810, 642465,
       642503, 614192, 661790, 661825, 661910, 662039, 662092, 683386,
       659472, 659600, 659694, 659753, 659914, 680426, 680812, 757887,
       758032, 758360, 782329, 782581, 782661, 782821, 782838, 782882,
       782883, 783197, 783322, 776150, 799188, 799221, 799317, 799360,
       799493, 799560, 799665, 799673, 799674, 799686, 799885, 799944,
       799949, 799972, 823354, 823362, 823447, 823454, 823493, 823557,
       823690, 823724, 823796, 823832, 823841, 823918, 823998, 824034,
       824065, 824112, 824190], dtype='object')
# this takes out the rows that are in nebular_rerun
good_fits_only = cigale_df_to_save.loc[cigale_df_to_save['id'].isin(good_fits)]
good_fits_only.to_csv('tentative_data.txt', sep='\t', index=False)